<a href="https://colab.research.google.com/github/ponytailsud2/NLP_roBERT_downey_jr/blob/main/Use_Thai__Depression_wanchangberta_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Tools

In [36]:
# Set timming

!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.36 s (started: 2022-11-09 07:26:31 +00:00)


In [37]:
# Install thai2transformer 
!pip install torch==1.5.0 torchtext==0.4.0 torchvision==0.6.0
!pip install transformers==3.5.0 thai2transformers==0.1.2
!pip install torch==1.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.13.0-cp37-cp37m-manylinux1_x86_64.whl (890.2 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.5.0
    Uninstalling torch-1.5.0:
      Successfully uninstalled torch-1.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.6.0 requires torch==1.5.0, but you have torch 1.13.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.13.0 which is incompatible.
fastai 2.7.10 requires torchvision>=0.8.2, but you have torchvision 0.6.0 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.5.0-cp37-cp37m-manylinux1_x86_64.whl (752.0 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.5.0 which is incompatible.
pytorch-lightning 1.8.0.post1 requires torch>=1.9.*, but you have torch 1.5.0 which is incompatible.
lightning-lite 1.8.0.post1 requires torch>=1.9.*, but you have torch 1.5.0 which is incompatible.
fastai 2.7.10 requires torch<1.14,>=1.7, but you have torch 1.5.0 which is incompatible.
fastai 2.7.10 requires torchvision>=0.8.2, but you have torchvision 0.6.0 which is incompatible.


time: 1min 6s (started: 2022-11-09 07:26:34 +00:00)


In [38]:
import torch
from tqdm.notebook import tqdm
from tqdm.auto import tqdm

from transformers import BertTokenizer
from transformers import DistilBertTokenizerFast
from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

import pandas as pd
import random
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, mean_squared_error, confusion_matrix, accuracy_score

time: 1.38 ms (started: 2022-11-09 07:27:41 +00:00)


In [39]:
#datasets
from datasets import load_dataset

#transformers
from transformers import (
    CamembertTokenizer,
    AutoTokenizer,
    AutoModel,
    AutoModelForMaskedLM,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)

#thai2transformers
import thai2transformers
from thai2transformers.preprocess import process_transformers
from thai2transformers.metrics import (
    classification_metrics, 
    multilabel_classification_metrics,
)
from thai2transformers.tokenizers import (
    ThaiRobertaTokenizer,
    ThaiWordsNewmmTokenizer,
    ThaiWordsSyllableTokenizer,
    FakeSefrCutTokenizer,
    SEFR_SPLIT_TOKEN
)

time: 1.06 ms (started: 2022-11-09 07:27:41 +00:00)


In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 3.1 s (started: 2022-11-09 07:27:41 +00:00)


### Import Data and select topic

In [41]:
#import data

## Original data 0409 training
# data_path = "/content/drive/MyDrive/thai_depression/data_out_0409_HAMD4_label.csv"
# data_path = "/content/drive/MyDrive/thai_depression/data_out_0409_HAMD4_label-1.csv"
# data_path = "/content/drive/MyDrive/thai_depression/data_out_0409_HAMD4_label-2.csv"
# data_path = "/content/drive/MyDrive/thai_depression/data_out_0409_HAMD4_label-3.csv"
# data_path = "/content/drive/MyDrive/thai_depression/data_out_0409_HAMD4_label-4.csv"

## Original data 0409+2108 training
# data_path = "/content/drive/MyDrive/thai_depression/hamd4-sleep-0408-2108/data_out_0409-2108_HAMD4_label.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hamd4-sleep-0408-2108/data_out_0409-2108_HAMD4_label-1.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hamd4-sleep-0408-2108/data_out_0409-2108_HAMD4_label-2.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hamd4-sleep-0408-2108/data_out_0409-2108_HAMD4_label-3.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hamd4-sleep-0408-2108/data_out_0409-2108_HAMD4_label-4.csv"

## Backtranslate 1 times data 0409+2108 training
# data_path = "/content/drive/MyDrive/thai_depression/hame-sleep-0408-2108-bc1/train-BC1_0409-2108_HAMD4_label.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hame-sleep-0408-2108-bc1/train-BC1_0409-2108_HAMD4_label-1.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hame-sleep-0408-2108-bc1/train-BC1_0409-2108_HAMD4_label-2.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hame-sleep-0408-2108-bc1/train-BC1_0409-2108_HAMD4_label-3.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hame-sleep-0408-2108-bc1/train-BC1_0409-2108_HAMD4_label-4.csv"

## Backtranslate 2 times data 0409+2108 training
# data_path = "/content/drive/MyDrive/thai_depression/hamd-sleep-0409-2108-bc2/train-BC2_0409-2108_HAMD4_label.csv"
data_path = "/content/drive/MyDrive/thai_depression/hamd-sleep-0409-2108-bc2/train-BC2_0409-2108_HAMD4_label-1.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hamd-sleep-0409-2108-bc2/train-BC2_0409-2108_HAMD4_label-2.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hamd-sleep-0409-2108-bc2/train-BC2_0409-2108_HAMD4_label-3.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hamd-sleep-0409-2108-bc2/train-BC2_0409-2108_HAMD4_label-4.csv"

## Backtranslate 1 times and balance data 0409+2108 training
# data_path = "/content/drive/MyDrive/thai_depression/hamd-sleep-0409-2108-bc1-balance/train-BC1-balance_0409-2108_HAMD4_label.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hamd-sleep-0409-2108-bc1-balance/train-BC1-balance_0409-2108_HAMD4_label-1.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hamd-sleep-0409-2108-bc1-balance/train-BC1-balance_0409-2108_HAMD4_label-2.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hamd-sleep-0409-2108-bc1-balance/train-BC1-balance_0409-2108_HAMD4_label-3.csv"
# data_path = "/content/drive/MyDrive/thai_depression/hamd-sleep-0409-2108-bc1-balance/train-BC1-balance_0409-2108_HAMD4_label-4.csv"

## Testing dataset
data_path_test = "/content/drive/MyDrive/thai_depression/data_out_0708_HAMD4_label.csv"

## Add train/test into dataframe
df_main = pd.read_csv(data_path)
df_main_test = pd.read_csv(data_path_test)
df_main = df_main.drop(['Unnamed: 0'], axis=1)
df_main.head()

,user_id,word,เลขประจำตัวอาสาสมัคร,ประเภทอาสาสมัคร,ระบบปฎิบัติการโทรศัพท์มือถือที่ใช้,รุ่นมือถือที่ใช้ร่วมกิจกรรม,อายุ (ปี),เพศ,ศาสนา,ภูมิลำเนา,...,9,10,11,12,13,14,15,16,17,คะแนนรวม HAM-D
0,164,จริงจริงเป็นเกือบทุกวันเลยค่ะเพราะว่าตอนกลางคื...,NaN,ไม่เป็นโรคซึมเศร้า,iOS,iPhone 13,21,หญิง,พุทธ,ภาคกลาง,...,1,1,1,0,1,1,1,0,0,9
1,165,นานนานทีเป็นค่ะแต่ว่าเท่าที่ทำrileyจบอาการตัวเ...,NaN,มีภาวะซึมเศร้า,iOS,iPhone 13,33,หญิง,พุทธ,ภาคเหนือ,...,1,1,1,0,0,1,0,0,0,7
2,166,เวลาถ้าเที่ยวหรือมีเรื่องผิดปกติขึ้นมามันจะดีด...,NaN,ไม่เป็นโรคซึมเศร้า,iOS,iPad air gen3,21,ชาย,พุทธ,ภาคกลาง,...,0,1,2,0,1,1,1,0,0,10
3,167,ก็คือเหมือนว่าจะสะดุ้งตื่นกลางดึกหรือว่าในช่วง...,NaN,ไม่มีข้อมูล,ไม่มีข้อมูล,ไม่มีข้อมูล,24,หญิง,พุทธ,ภาคเหนือ,...,0,1,1,0,0,0,0,0,0,4
4,168,การนอนก็คือนอนไม่หลับเลยพอหลับตาปั๊บสมองมันไม่...,NaN,ไม่มีข้อมูล,ไม่มีข้อมูล,ไม่มีข้อมูล,24,ชาย,ไม่มีนับถือศาสนา,ภาคกลาง,...,0,2,2,1,2,0,0,0,0,12


time: 652 ms (started: 2022-11-09 07:27:44 +00:00)


In [42]:
df_main_test = df_main_test.drop(['Unnamed: 0'], axis=1)
df_main_test.head()

,user_id,word,อายุ (ปี),เพศ,ศาสนา,ภูมิลำเนา,สถานภาพ,จำนวนบุตร-ธิดา,ระดับการศึกษา,ตำแหน่งงาน,...,9,10,11,12,13,14,15,16,17,คะแนนรวม HAM-D
0,136,NaN,25,หญิง,พุทธ,ภาคกลาง,โสด,0,ปริญญาตรี,พนักงานบริษัทเอกชน,...,1,0,1,0,0,0,0,0,0,3
1,135,ก็สลับด้านนี้และครั้งครับยังน่าจะเป็นวันที่แบบ...,27,ชาย,พุทธ,ภาคกลาง,โสด,0,ปริญญาตรี,พนักงานบริษัทเอกชน,...,0,0,0,0,1,0,0,0,0,2
2,132,ช่วงนี้ก็เป็นบ่อยเพราะว่าเหมือนมีกิจกรรมให้ทำเ...,43,ชาย,พุทธ,ภาคกลาง,โสด,2,สูงกว่าปริญญาตรี,พนักงานบริษัทเอกชน,...,0,0,0,0,0,0,0,0,0,1
3,130,คือจะถึงหกโมงเช้าตามนี้เลยมีslotเวลาในการนอนเป...,25,ชาย,ไม่มีนับถือศาสนา,ภาคตะวันออกเฉียงเหนือ,โสด,0,ปริญญาตรี,พนักงานบริษัทเอกชน,...,0,0,1,0,1,0,0,0,0,3
4,134,ครับเดี๋ยวผมยังนอนได้ปกตินะฮะโชคดีที่เป็นคนนอน...,31,ชาย,ไม่มีนับถือศาสนา,ภาคกลาง,สมรส,0,ปริญญาตรี,พนักงานบริษัทเอกชน,...,0,1,2,0,1,1,0,0,0,5


time: 16 ms (started: 2022-11-09 07:27:45 +00:00)


In [43]:
## Select column in train to predict
df = df_main.filter(['user_id','word','4'], axis=1)
df = df.dropna()
df.head()

,user_id,word,4
0,164,จริงจริงเป็นเกือบทุกวันเลยค่ะเพราะว่าตอนกลางคื...,1
1,165,นานนานทีเป็นค่ะแต่ว่าเท่าที่ทำrileyจบอาการตัวเ...,0
2,166,เวลาถ้าเที่ยวหรือมีเรื่องผิดปกติขึ้นมามันจะดีด...,1
3,167,ก็คือเหมือนว่าจะสะดุ้งตื่นกลางดึกหรือว่าในช่วง...,0
4,168,การนอนก็คือนอนไม่หลับเลยพอหลับตาปั๊บสมองมันไม่...,2


time: 9.02 ms (started: 2022-11-09 07:27:45 +00:00)


In [44]:
## Select column int test to predict
df_test = df_main_test.filter(['user_id','word','4'], axis=1)
df_test = df_test.dropna()
df_test.head()

,user_id,word,4
1,135,ก็สลับด้านนี้และครั้งครับยังน่าจะเป็นวันที่แบบ...,1
2,132,ช่วงนี้ก็เป็นบ่อยเพราะว่าเหมือนมีกิจกรรมให้ทำเ...,1
3,130,คือจะถึงหกโมงเช้าตามนี้เลยมีslotเวลาในการนอนเป...,0
4,134,ครับเดี๋ยวผมยังนอนได้ปกตินะฮะโชคดีที่เป็นคนนอน...,0
5,127,สะดุ้งตื่นชักประมาณช่วงตีหนึ่งตีสองเป็นทุกวันแ...,0


time: 8.06 ms (started: 2022-11-09 07:27:45 +00:00)


In [45]:
## Rename column train
df = df.rename(columns={'4': 'label'})
df = df.rename(columns={'word': 'value'})
df.head()

,user_id,value,label
0,164,จริงจริงเป็นเกือบทุกวันเลยค่ะเพราะว่าตอนกลางคื...,1
1,165,นานนานทีเป็นค่ะแต่ว่าเท่าที่ทำrileyจบอาการตัวเ...,0
2,166,เวลาถ้าเที่ยวหรือมีเรื่องผิดปกติขึ้นมามันจะดีด...,1
3,167,ก็คือเหมือนว่าจะสะดุ้งตื่นกลางดึกหรือว่าในช่วง...,0
4,168,การนอนก็คือนอนไม่หลับเลยพอหลับตาปั๊บสมองมันไม่...,2


time: 7.72 ms (started: 2022-11-09 07:27:45 +00:00)


In [46]:
## Rename column test
df_test = df_test.rename(columns={'4': 'label'})
df_test = df_test.rename(columns={'word': 'value'})
df_test.head()

,user_id,value,label
1,135,ก็สลับด้านนี้และครั้งครับยังน่าจะเป็นวันที่แบบ...,1
2,132,ช่วงนี้ก็เป็นบ่อยเพราะว่าเหมือนมีกิจกรรมให้ทำเ...,1
3,130,คือจะถึงหกโมงเช้าตามนี้เลยมีslotเวลาในการนอนเป...,0
4,134,ครับเดี๋ยวผมยังนอนได้ปกตินะฮะโชคดีที่เป็นคนนอน...,0
5,127,สะดุ้งตื่นชักประมาณช่วงตีหนึ่งตีสองเป็นทุกวันแ...,0


time: 8.44 ms (started: 2022-11-09 07:27:45 +00:00)


### Label train test

In [47]:
#count label train
df['label'].value_counts()

0    93
1    57
2    27
Name: label, dtype: int64

time: 6.46 ms (started: 2022-11-09 07:27:45 +00:00)


In [48]:
#count label test
df_test['label'].value_counts()

0    11
1     9
2     1
Name: label, dtype: int64

time: 4.5 ms (started: 2022-11-09 07:27:45 +00:00)


In [49]:
#encoding label

possible_labels = df.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    if possible_label == 0 :
      label_dict[possible_label] = 0
    elif possible_label == 1 :
      label_dict[possible_label] = 1
    elif possible_label == 2 :
      label_dict[possible_label] = 1
    # elif possible_label == 3 :
    #   label_dict[possible_label] = 3
    else :
      print("Cannot do that")
    # print(index)
    # print("posible label : ", possible_label)
    # label_dict[possible_label] = index
label_dict

{1: 1, 0: 0, 2: 1}

time: 4.07 ms (started: 2022-11-09 07:27:45 +00:00)


In [50]:
df['label'] = df.label.replace(label_dict)
df.head()

,user_id,value,label
0,164,จริงจริงเป็นเกือบทุกวันเลยค่ะเพราะว่าตอนกลางคื...,1
1,165,นานนานทีเป็นค่ะแต่ว่าเท่าที่ทำrileyจบอาการตัวเ...,0
2,166,เวลาถ้าเที่ยวหรือมีเรื่องผิดปกติขึ้นมามันจะดีด...,1
3,167,ก็คือเหมือนว่าจะสะดุ้งตื่นกลางดึกหรือว่าในช่วง...,0
4,168,การนอนก็คือนอนไม่หลับเลยพอหลับตาปั๊บสมองมันไม่...,1


time: 7.53 ms (started: 2022-11-09 07:27:45 +00:00)


In [51]:
df_test['label'] = df_test.label.replace(label_dict)
df_test.head()

,user_id,value,label
1,135,ก็สลับด้านนี้และครั้งครับยังน่าจะเป็นวันที่แบบ...,1
2,132,ช่วงนี้ก็เป็นบ่อยเพราะว่าเหมือนมีกิจกรรมให้ทำเ...,1
3,130,คือจะถึงหกโมงเช้าตามนี้เลยมีslotเวลาในการนอนเป...,0
4,134,ครับเดี๋ยวผมยังนอนได้ปกตินะฮะโชคดีที่เป็นคนนอน...,0
5,127,สะดุ้งตื่นชักประมาณช่วงตีหนึ่งตีสองเป็นทุกวันแ...,0


time: 7.04 ms (started: 2022-11-09 07:27:45 +00:00)


In [52]:
#count label train
df['label'].value_counts()

0    93
1    84
Name: label, dtype: int64

time: 4.35 ms (started: 2022-11-09 07:27:45 +00:00)


In [53]:
#count label test
df_test['label'].value_counts()

0    11
1    10
Name: label, dtype: int64

time: 4.17 ms (started: 2022-11-09 07:27:45 +00:00)


In [54]:
df.to_csv('label_text.csv', index = True)

time: 5.82 ms (started: 2022-11-09 07:27:45 +00:00)


### Split train / test

In [55]:
#solve umbalance data
# column_name = ['Number', 'Participant_ID']
# X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  # df.label.values, 
                                                  # test_size=0.15, 
                                                  # random_state=62, 
                                                  # stratify=df.label.values)

X_train = df.index.values
X_val = df_test.index.values

y_train = df.label.values
y_val = df_test.label.values

df['data_type'] = ['not_set']*df.shape[0]
# df_test['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df_test.loc[X_val, 'data_type'] = 'val'
df2 = df_test.loc[X_val,'user_id']
df.groupby(['label', 'label', 'data_type']).count()

,,,user_id,value
label,label,data_type,,
0,0,train,93,93
1,1,train,84,84


time: 14.3 ms (started: 2022-11-09 07:27:45 +00:00)


In [56]:
df_test.groupby(['label', 'label', 'data_type']).count()

,,,user_id,value
label,label,data_type,,
0,0,val,11,11
1,1,val,10,10


time: 8.61 ms (started: 2022-11-09 07:27:45 +00:00)


In [57]:
df2 = df2.sort_index()

time: 1.3 ms (started: 2022-11-09 07:27:45 +00:00)


In [58]:
df2.to_csv('label_type.csv', index = False)

time: 5.68 ms (started: 2022-11-09 07:27:45 +00:00)


### Setting for training

In [59]:
## Build tokenizer and model
model_name = 'wangchanberta-base-wiki-spm'

model_name_hugging = f'airesearch/{model_name}'
tokenizers = {
    "bert-base-multilingual-uncased" : AutoTokenizer, 
    "Geotrend/bert-base-th-cased" : AutoTokenizer,
    'monsoon-nlp/bert-base-thai': AutoTokenizer,
    'airesearch/wangchanberta-base-att-spm-uncased': AutoTokenizer,
    'xlm-roberta-base': AutoTokenizer,
    'bert-base-multilingual-cased': AutoTokenizer,
    'airesearch/wangchanberta-base-wiki-newmm': ThaiWordsNewmmTokenizer,
    'airesearch/wangchanberta-base-wiki-ssg': ThaiWordsSyllableTokenizer,
    'airesearch/wangchanberta-base-wiki-sefr': FakeSefrCutTokenizer,
    'airesearch/wangchanberta-base-wiki-spm': ThaiRobertaTokenizer}

tokenizer = tokenizers[model_name_hugging].from_pretrained(
                model_name_hugging,
                revision='main',
                model_max_length=416,)
model = AutoModelForSequenceClassification.from_pretrained(model_name_hugging,num_labels=len(label_dict))


Some weights of the model checkpoint at airesearch/wangchanberta-base-wiki-spm were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-wiki-spm and are newly initialized: ['classifier.dense.wei

time: 13.3 s (started: 2022-11-09 07:27:45 +00:00)


In [60]:
#convert text into encoding form

encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].value.values, 
    add_special_tokens=True, #sequences will be encoded with the special tokens relative to their model
    return_attention_mask=True, #batching sequences together
    pad_to_max_length=True, # pad all the titles to certain maximum length
    max_length=512, #attention mask according to the specific tokenizer 
    return_tensors='pt' #return PyTorch
)

encoded_data_val = tokenizer.batch_encode_plus(
    df_test[df_test.data_type=='val'].value.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

# split the data into input_ids, attention_masks and labels
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_test[df_test.data_type=='val'].label.values)

#get training data and validation data
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


time: 153 ms (started: 2022-11-09 07:27:59 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [61]:
# DataLoader combines a dataset and a sampler, and provides an iterable over the given dataset
# RandomSampler for training and SequentialSampler for validation

batch_size = 3 #limited memory in my environment

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

time: 1.35 ms (started: 2022-11-09 07:27:59 +00:00)


In [62]:
## Optimize model

learning_rate = 1e-5
# epsilon = 1e-8
epochs = 10
# epochs = 5

# Ademw declide overfit
optimizer = AdamW(model.parameters(), lr=learning_rate)

# schedule with a learning rate that decreases linearly from the initial learning rate set
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

time: 2.1 ms (started: 2022-11-09 07:27:59 +00:00)


In [63]:
## Calculate score of model (function)

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    print("preds_flat : ", preds_flat)
    labels_flat = labels.flatten()
    print("labels_flat: ", labels_flat)
    f1_micro = f1_score(labels_flat, preds_flat, average='micro')
    f1_marcro = f1_score(labels_flat, preds_flat, average='macro')
    return f1_micro, f1_marcro

def mean_squear_error(preds,labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return mean_squared_error(labels_flat, preds_flat)

def confusion_metrix_cal(preds,labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return confusion_matrix(labels_flat, preds_flat)

def accurency_cal(preds,labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return accuracy_score(labels_flat, preds_flat)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

time: 3.01 ms (started: 2022-11-09 07:27:59 +00:00)


### Training

In [64]:
device = torch.device('cpu')

time: 638 µs (started: 2022-11-09 07:27:59 +00:00)


In [ ]:
seed_val = 19
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    # print(model)
    loss_val_total = 0
    predictions, true_vals, input_count = [], [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        # print(label_ids)
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    # print(predictions)
    # print("true_vals : ", true_vals)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        barch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       
        # print(batch)
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/depression/finetuned_BERT_epoch_{epoch}_4.model')
    # torch.save(model.state_dict(), f'/content/{epoch}_4.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1_micro, val_f1_marcro = f1_score_func(predictions, true_vals)
    means_sq = mean_squear_error(predictions, true_vals)
    confuse_me = confusion_metrix_cal(predictions, true_vals)
    accurency_sc = accurency_cal(predictions, true_vals)
    
    tqdm.write(f'Validation loss: {val_loss}')
    # tqdm.write(f'F1 Score (Weighted): {val_f1}')
    tqdm.write(f'F1 Score (micro): {val_f1_micro}')
    tqdm.write(f'F1 Score (marcro): {val_f1_marcro}')
    tqdm.write(f'MSE Score : {means_sq}')
    tqdm.write(f'Confuse Metrix : {confuse_me}')
    tqdm.write(f'Accurency Score : {accurency_sc}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/59 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.7720519324480477
preds_flat :  [1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1]
labels_flat:  [1 1 0 0 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 0 1]
Validation loss: 0.6535984788622174
F1 Score (micro): 0.6666666666666666
F1 Score (marcro): 0.6541176470588235
MSE Score : 0.3333333333333333
Confuse Metrix : [[9 2]
 [5 5]]
Accurency Score : 0.6666666666666666


Epoch 2:   0%|          | 0/59 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.6147970021781275
preds_flat :  [1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 1]
labels_flat:  [1 1 0 0 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 0 1]
Validation loss: 0.602410078048706
F1 Score (micro): 0.6190476190476191
F1 Score (marcro): 0.6111111111111112
MSE Score : 0.38095238095238093
Confuse Metrix : [[8 3]
 [5 5]]
Accurency Score : 0.6190476190476191


Epoch 3:   0%|          | 0/59 [00:00<?, ?it/s]